**Классификация изображений с помощью сверточных нейронных сетей**

В данном задании Вам необходимо разработать архитектуру сверточной ИНС, обеспечивающую наибольшую точность при ограничении на количество операций (FLOPs <= 0.707e6).
Заготовка кода для выполнения задания приведена выше. Вашей задачей будет заполнить пропущеные места, которые отмечены ключевым словом *None*.
Необходимая точность (accuracy) сети на датасете CIFAR100 - 30%
Желаемая точность (accuracy) сети на датасете CIFAR100 - 45%

In [1]:
!pip install keras-flops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 24.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 54.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.1.1
    Uninstalling typing_extensions-4.1.1:
      Successfully uninstalled typing_extensions-4.1.1
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: h5py
    Found existing installation: h5py 3.7.0
    Uninstalling h5py-3.7.0:
      Successfully uninstalled h5py-3.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
dask-cudf 21.10.1 requires cupy-cuda114, which is not installe

In [2]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_flops import get_flops
from tensorflow import keras
from keras import layers

In [3]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [4]:
# Выполните загрузку модели
(X_train, y_train), (X_val, y_val) = tf.keras.datasets.cifar100.load_data()

169017344/169001437 [==============================] - 2s 0us/step


In [5]:
# Преобразуйте метки классов в one_hot формат
y_train = (y_train == np.arange(CLASSES)).astype(np.float32)
y_val = (y_val == np.arange(CLASSES)).astype(np.float32)

In [6]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

In [8]:
# Задайте архитектуру модели
from keras.models import Model

inputs = keras.Input(shape=[32,32,3])
x = layers.Conv2D(16, (3, 3), strides=(2, 2), padding='same')(inputs)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('selu')(x)
x = layers.Dropout(0.01)(x)

x = layers.SeparableConvolution2D(32, (3, 3), strides=(1, 1), padding='same')(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('leaky_relu')(x)

x = layers.SeparableConvolution2D(64, (3, 3), strides=(2, 2), padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('leaky_relu')(x)

x = layers.SeparableConvolution2D(128, (3, 3), strides=(1, 1), padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('leaky_relu')(x)

x = layers.SeparableConvolution2D(128, (3, 3), strides=(2, 2), padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('leaky_relu')(x)

x = layers.Conv2D(64, (1, 1), padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("selu")(x)
x = layers.Dropout(0.2)(x)

x = layers.SeparableConvolution2D(256, (3, 3), strides=(1, 1), padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('leaky_relu')(x)

x = layers.SeparableConvolution2D(256, (3, 3), strides=(2, 2), padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('leaky_relu')(x)

x = layers.Flatten()(x)
x = layers.Activation('selu')(x)
out = layers.Dense(CLASSES, activation='softmax')(x)

model = Model(inputs, out)

In [9]:
# вычисление количества операций
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")


=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math behind it.

Profi

2023-01-25 00:52:36.649710: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-25 00:52:36.650117: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-01-25 00:52:36.650283: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2023-01-25 00:52:36.650763: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-25 00:52:36.651189: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-25 00:52:36.651551: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] su

In [10]:
# вывод краткой информации о модели
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 16)        448       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 16)          0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 8, 8, 16)          64        
_________________________________________________________________
activation_9 (Activation)    (None, 8, 8, 16)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 16)          0         
_________________________________________________________________
separable_conv2d_6 (Separabl (None, 8, 8, 32)          688 

In [12]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model.compile(
    optimizer=tf.keras.optimizers.Adam(#learning_rate = 0.01
        learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
            0.006,
            decay_steps=1500, # One epoch 1563 0.95
            decay_rate=0.95,
            staircase=True)
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy'])

In [13]:
# обучения модели
model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
        
    ],
    use_multiprocessing=True,
    workers=8,
    epochs=256
)

2023-01-25 00:54:08.375297: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/256


2023-01-25 00:54:10.481775: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


391/391 [==============================] - 11s 8ms/step - loss: 3.7772 - accuracy: 0.1116 - val_loss: 4.0888 - val_accuracy: 0.0874
Epoch 2/256
391/391 [==============================] - 3s 9ms/step - loss: 3.3530 - accuracy: 0.1763 - val_loss: 3.7763 - val_accuracy: 0.1695
Epoch 3/256
391/391 [==============================] - 3s 7ms/step - loss: 3.1605 - accuracy: 0.2135 - val_loss: 3.5699 - val_accuracy: 0.1963
Epoch 4/256
391/391 [==============================] - 3s 7ms/step - loss: 3.0359 - accuracy: 0.2353 - val_loss: 3.1858 - val_accuracy: 0.2301
Epoch 5/256
391/391 [==============================] - 3s 7ms/step - loss: 2.9417 - accuracy: 0.2571 - val_loss: 3.3711 - val_accuracy: 0.2084
Epoch 6/256
391/391 [==============================] - 3s 8ms/step - loss: 2.8680 - accuracy: 0.2702 - val_loss: 2.9630 - val_accuracy: 0.2690
Epoch 7/256
391/391 [==============================] - 3s 7ms/step - loss: 2.8076 - accuracy: 0.2811 - val_loss: 3.3343 - val_accuracy: 0.2288
Epoch 8/25